# Михнева Анна, 211

In [1]:
from collections import Counter
import nltk
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from pymystem3 import Mystem
m = Mystem()
import json
import re
import os

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/anna_mikhneva_/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## собираем корпус
#### основные особенности:
1.Причастия, деепричастия (преувеличенные, испускаемое, обнажая и тд) <br>
2.Имена собственные: случаи, где они уже превратились в имена существительные (гансов, иванов), аббревиатуры и сокращения (мфти, росторгмонтаж) <br>
3.Непопулярная лексика, которая может отсутствовать в словаре (корпускулярное, хаб) <br>
4.Омонимия (лень - существительное/наречие, бегом - существительное/наречие и тд) <br>
5.Разные виды числительных (порядковые, количественные, собирательные) <br>
6.Местоимения через дефис (что-то, какие-то) <br>
7.Неграмматичные или непривычные словоизменения (два дни, деревяннее) <br>

In [2]:
words = []
with open('корпус.txt', 'r') as file:
    for text in file:
        for word in nltk.word_tokenize(text.lower()):
            if word.isalpha() or '-' in word:
                words.append(word)
print(words)
len(words)

['нам', 'учиться', 'лодырям', 'что-то', 'лень', 'на', 'коньках', 'катаемся', 'целый', 'день', 'горе', 'однако', 'в', 'том', 'что', 'подобный', 'же', 'бред', 'осуществлялся', 'в', 'реальности', 'в', 'жизни', 'в', 'судьбах', 'миллионов', 'гансов', 'или', 'иванов', 'излучение', 'электронное', 'или', 'позитронное', 'корпускулярное', 'ионизирующее', 'излучение', 'с', 'непрерывным', 'энергетическим', 'спектром', 'испускаемое', 'при', 'ядерных', 'превращениях', 'теперь', 'вниз', 'бегом', 'по', 'набережной', 'шагом', 'марш', 'два', 'дни', 'и', 'две', 'ночи', 'спал', 'петро', 'без', 'просыпу', 'но', 'двое', 'молодых', 'людей', 'сопровождавшие', 'больного', 'не', 'смеялись', 'они', 'едва', 'держались', 'на', 'ногах', 'после', 'двух', 'суток', 'проведенных', 'без', 'сна', 'наедине', 'с', 'безумным', 'которого', 'они', 'только', 'что', 'привезли', 'по', 'железной', 'дороге', 'сверх', 'изорванного', 'во', 'время', 'припадка', 'в', 'клочья', 'серого', 'платья', 'куртка', 'из', 'грубой', 'парусины', 

258

## Выбор разметки
Я выбрала разметку pymorphy, как наиболее подробную и подходящую к моему корпусу и менее подвижную для изменений. Однако оттуда я убрала различия в полных/неполных прилагательных и причастиях и тег для инфинитивов, так как эти теги отсутствуют совсем в разметке universal dependencies. <br>
<br>
Ниже памятка для меня при составлении функции

### мои теги
NOUN, VERB, PRON, PROPN, ADVB, NUMR, ADJ, CONJ, PART, INTJ, GRND, PRT, PREP
### tagger11 - pymorphy
NPRO - PRON, INFN - VERB, ADJF/ADJS - ADJ, PRCL - PART, PRTF/PRTS - PRT
### tagger2 - mystem
A - ADJ,
ADV - ADVB,
ADVPRO/APRO/SPRO - PRON,
ANUM/NUM - NUMR,
PR - PREP,
S - NOUN,
V - VERB, добавить причастия и деепричастия (отмечает как глаголы)
### tagger3 - stanza
SCONJ/CCONJ - CONJ, AUX - VERB, NUM -NUMR, ADP - PREP, ADV - ADVB, DET - PRON, добавить причастия и деепричастия (отмечает как глаголы)

In [3]:
def tagging(tagger):
    for index, key in enumerate(tagger):
        if re.match('.+PRO', key) or key == 'DET':
            tagger[index] = 'PRON'
        elif re.match('ADJ.', key) or key == 'A':
            tagger[index] = 'ADJ'
        elif key == 'S':
            tagger[index] = 'NOUN'
        elif key == 'V' or key == 'AUX' or key == 'INFN':
            tagger[index] = 'VERB'
        elif key == 'ADV':
            tagger[index] = 'ADVB'
        elif re.match('.?NUM', key):
            tagger[index] = 'NUMR'
        elif re.match('.CONJ', key):
            tagger[index] = 'CONJ'
        elif key == 'PRCL':
            tagger[index] = 'PART'
        elif key == 'PR' or key == 'ADP':
            tagger[index] = 'PREP'
        elif key == None:
            tagger[index] = 'unknown'
        elif re.match('PRT.', key):
            tagger[index] = 'PRT'
    return tagger

## Проверка первого теггера

In [4]:
tagger1 = []
for i in words:
    p = morph.parse(i)[0]
    if p.tag.POS == None:
        tagger1.append('unknown')
    else:
        tagger1.append(p.tag.POS)
a = '\n'.join(tagger1)
# pymorphy запрещает выполнять функцию и менять теги в списке, поэтому пришлось записать список в файл и прочитать, чтобы функция по замене работала
tagger11 = []
with open ('теггер.txt', 'w') as f:
    f.write(f'{a}')
with open ('теггер.txt', 'r') as file:
    content = file.readlines()
    for word in content:
        tagger11.append(word.strip())
print(tagging(tagger11))
len(tagger11)

['PRON', 'VERB', 'NOUN', 'ADVB', 'NOUN', 'PREP', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'CONJ', 'PREP', 'ADJ', 'CONJ', 'ADJ', 'PART', 'NOUN', 'VERB', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'ADJ', 'CONJ', 'ADJ', 'ADJ', 'PRT', 'NOUN', 'PREP', 'ADJ', 'ADJ', 'NOUN', 'PRT', 'PREP', 'ADJ', 'NOUN', 'ADVB', 'ADVB', 'ADVB', 'PREP', 'NOUN', 'NOUN', 'INTJ', 'NUMR', 'NOUN', 'CONJ', 'NUMR', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NUMR', 'ADJ', 'NOUN', 'PRT', 'ADJ', 'PART', 'VERB', 'PRON', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PREP', 'NUMR', 'NOUN', 'PRT', 'PREP', 'NOUN', 'ADVB', 'PREP', 'ADJ', 'ADJ', 'PRON', 'ADVB', 'CONJ', 'VERB', 'PREP', 'ADJ', 'NOUN', 'PREP', 'PRT', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'VERB', 'PRON', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PRT', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'PART', 'PART', 'VERB', 'ADJ', 'NOUN', 

258

## Проверка второго теггера

In [5]:
tagger2 = []
for i in m.analyze(''.join(text)):
    # print(i)
    if len(i) != 1:
        if i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[уюая]щ[аоеиы][йяе]?$', i['text']):
            tagger2.append('PRT')
        elif i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[оеи]м[аоеиы][йяе]?$', i['text']):
            tagger2.append('PRT')
        elif i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[в]?ш[аоеи][йяе]?$', i['text']):
            tagger2.append('PRT')
        elif i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[eё]?н?н[аоеиы][йяе]?$', i['text']):
            tagger2.append('PRT')
        elif i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[в]$', i['text']):
            tagger2.append('GRND')
        elif i['analysis'][0]['gr'].split(',')[0].split('=')[0] == 'V' and re.search(r'[^ь][^c][ая]$', i['text']):
            tagger2.append('GRND')
        else:
            tagger2.append(i['analysis'][0]['gr'].split(',')[0].split('=')[0])
print(tagging(tagger2))
len(tagger2)

['PRON', 'VERB', 'NOUN', 'PRON', 'NOUN', 'PREP', 'NOUN', 'GRND', 'ADJ', 'NOUN', 'NOUN', 'CONJ', 'PREP', 'PRON', 'CONJ', 'ADJ', 'PART', 'NOUN', 'GRND', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'ADJ', 'CONJ', 'ADJ', 'ADJ', 'PRT', 'NOUN', 'PREP', 'ADJ', 'ADJ', 'NOUN', 'PRT', 'PREP', 'ADJ', 'NOUN', 'ADVB', 'ADVB', 'ADVB', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'NUMR', 'NOUN', 'CONJ', 'NUMR', 'NOUN', 'VERB', 'NOUN', 'PREP', 'NOUN', 'CONJ', 'NUMR', 'ADJ', 'NOUN', 'PRT', 'NOUN', 'PART', 'VERB', 'PRON', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PREP', 'NUMR', 'NOUN', 'VERB', 'PREP', 'NOUN', 'ADVB', 'PREP', 'ADJ', 'PRON', 'PRON', 'PART', 'CONJ', 'VERB', 'PREP', 'ADJ', 'NOUN', 'PREP', 'VERB', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'GRND', 'PRON', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'PRON', 'PRON', 'VERB', 'ADJ', 'NO

258

## Проверка третьего теггера

In [6]:
import stanza

nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos')


2023-10-07 22:36:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-07 22:36:19 INFO: Loading these models for language: ru (Russian):
| Processor | Package          |
--------------------------------
| tokenize  | syntagrus        |
| pos       | syntagrus_charlm |

2023-10-07 22:36:19 INFO: Using device: cuda
2023-10-07 22:36:19 INFO: Loading: tokenize
2023-10-07 22:36:31 INFO: Loading: pos
2023-10-07 22:36:34 INFO: Done loading processors!


In [7]:
tagger3 = []
doc = nlp(text)
for sent in doc.sentences:
    for word in sent.words:
        if word.upos == 'PUNCT' or word.upos == 'SYM':
            continue
        elif word.upos == 'VERB' and re.search(r'[уюая]щ[аоеиы][йяе]?$', word.text):
            tagger3.append('PRT')
        elif word.upos == 'VERB' and re.search(r'[оеи]м[аоеиы][йяе]?$', word.text):
            tagger3.append('PRT')
        elif word.upos == 'VERB' and re.search(r'[в]?ш[аоеи][йяе]?$', word.text):
            tagger3.append('PRT')
        elif word.upos == 'VERB' and re.search(r'[eё]?н?н[аоеиы][йяе]?$', word.text):
            tagger3.append('PRT')
        elif word.upos == 'VERB' and re.search(r'[в]$', word.text):
            tagger3.append('GRND')
        elif word.upos == 'VERB' and re.search(r'[^ь][^c][ая]$', word.text):
            tagger3.append('GRND')
        else:
            tagger3.append(word.upos)
print(tagging(tagger3))
len(tagger3)

['PRON', 'VERB', 'NOUN', 'PRON', 'NOUN', 'PREP', 'NOUN', 'GRND', 'ADJ', 'NOUN', 'NOUN', 'ADVB', 'PREP', 'PRON', 'CONJ', 'ADJ', 'PART', 'NOUN', 'GRND', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'ADJ', 'CONJ', 'ADJ', 'ADJ', 'PRT', 'NOUN', 'PREP', 'ADJ', 'ADJ', 'NOUN', 'PRT', 'PREP', 'ADJ', 'NOUN', 'ADVB', 'ADVB', 'ADVB', 'PREP', 'NOUN', 'NOUN', 'NOUN', 'NUMR', 'NOUN', 'CONJ', 'NUMR', 'NOUN', 'VERB', 'PROPN', 'PREP', 'NOUN', 'CONJ', 'NUMR', 'ADJ', 'NOUN', 'PRT', 'NOUN', 'PART', 'VERB', 'PRON', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PREP', 'NUMR', 'NOUN', 'VERB', 'PREP', 'NOUN', 'ADVB', 'PREP', 'ADJ', 'PRON', 'PRON', 'PART', 'PRON', 'VERB', 'PREP', 'ADJ', 'NOUN', 'PREP', 'VERB', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'GRND', 'PRON', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PRT', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'PRON', 'PRON', 'VERB', 'ADJ', 'N

258

### Достаем нашу разметку из файла

In [8]:
import pandas as pd

read_file = pd.read_excel('tagged_corpus.xlsx')
read_file.to_csv('tagged_corpus.csv', index = None, header=True)

In [9]:
my_tags = []
import csv
with open('tagged_corpus.csv') as csv_file:
    reader = csv.DictReader(csv_file, delimiter=",")
    for row in reader:
        my_tags.append(row['tag'])
print(my_tags)
len(my_tags)

['PRON', 'VERB', 'NOUN', 'PRON', 'ADVB', 'PREP', 'NOUN', 'VERB', 'ADJ', 'NOUN', 'NOUN', 'CONJ', 'PREP', 'PRON', 'PRON', 'ADJ', 'PART', 'NOUN', 'VERB', 'PREP', 'NOUN', 'PREP', 'NOUN', 'PREP', 'NOUN', 'NUMR', 'NOUN', 'CONJ', 'NOUN', 'NOUN', 'ADJ', 'CONJ', 'ADJ', 'ADJ', 'PRT', 'NOUN', 'PREP', 'ADJ', 'ADJ', 'NOUN', 'PRT', 'PREP', 'ADJ', 'NOUN', 'ADVB', 'ADVB', 'ADVB', 'PREP', 'NOUN', 'ADVB', 'NOUN', 'NUMR', 'NOUN', 'CONJ', 'NUMR', 'NOUN', 'VERB', 'PROPN', 'PREP', 'ADVB', 'CONJ', 'NUMR', 'ADJ', 'NOUN', 'PRT', 'NOUN', 'PART', 'VERB', 'PRON', 'ADVB', 'VERB', 'PREP', 'NOUN', 'PREP', 'NUMR', 'NOUN', 'PRT', 'PREP', 'NOUN', 'ADVB', 'PREP', 'ADJ', 'PRON', 'PRON', 'ADVB', 'PRON', 'VERB', 'PREP', 'ADJ', 'NOUN', 'PREP', 'PRT', 'PREP', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'PREP', 'ADJ', 'NOUN', 'VERB', 'PRON', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'NOUN', 'ADJ', 'NOUN', 'NOUN', 'PREP', 'NOUN', 'PRT', 'NOUN', 'NOUN', 'CONJ', 'NOUN', 'PRON', 'PART', 'VERB', 'ADJ', 'NOU

258

In [10]:
from sklearn.metrics import accuracy_score

In [11]:
accuracy_score(tagger11, my_tags)

0.8643410852713178

In [12]:
accuracy_score(tagger2, my_tags)

0.8643410852713178

In [13]:
accuracy_score(tagger3, my_tags)

0.8875968992248062

In [14]:
accuracy_score(tagger11, tagger2) #просто проверила для себя, совпадают ли ошибки. Вывод: не совпадают

0.8527131782945736

### Вывод: на моем корпусе наилучший теггер для русского языка получился stanza

## Сложная часть 4

#### Шаблоны (1-3 для биграмм, 4 для трехграмм)
##### у меня английский корпус для дз1 с отзывами на фильмы
1. no + NOUN (случаи с no plot, no experience, no suspence, no balance, no tense) для негативных отзывов
2. lack + PREP (случаи с lack of smth, what we lack in this film is...) для негативных отзывов
3. well + VERB (well filmed/done/acted/known/rated) для позитивных отзывов
4. not + worth + VERB/it (not worth watching/seeing/experiencing/it) для негативных отзывов


In [15]:
from nltk.tag import pos_tag
nltk.download('universal_tagset')

[nltk_data] Downloading package universal_tagset to
[nltk_data]     /home/anna_mikhneva_/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


True

#### Функция для негативных отзывов

In [16]:
def chunker_neg(negative, n=2):
    output = []
    fin_out = []
    for i in range(len(negative)- n+1):
        output.append(negative[i:i+n])
    for bigram in output:
        check = bigram[1]
        ppp = pos_tag(bigram, tagset='universal')
        if ppp[0][0] == 'no' and ppp[1][1] == 'NOUN':
            fin_out.append(bigram)
        elif ppp[0][0] == 'lack' and ppp[1][1] == 'ADP':
            fin_out.append(bigram)
    return fin_out

#### Функция для позитивных отзывов

In [17]:
def chunker_pos(positive, n=2):
    output = []
    fin_out = []
    for i in range(len(positive)- n+1):
        output.append(positive[i:i+n])
    for bigram in output:
        check = bigram[1]
        ppp = pos_tag(bigram, tagset='universal')
        if ppp[0][0] == 'well' and ppp[1][1] == 'VERB':
            fin_out.append(bigram)
    return fin_out

### 5 ячеек ниже из прошлой домашки

In [18]:
negative = []
for filename in os.listdir('neg_test'):
    neg = os.path.join('neg_test', filename)
    if os.path.isfile(neg) and filename.endswith('.txt'):
        with open(neg, 'r') as fileneg:
            for inf in fileneg:
                for slovo in nltk.word_tokenize(inf.lower()):
                    if slovo.isalpha():
                        finword = morph.parse(slovo)[0]
                        negative.append(finword[0])
print(negative)
chunker_neg(negative) #список из шаблонов 1 и 2

['once', 'again', 'costner', 'has', 'dragged', 'out', 'a', 'movie', 'for', 'far', 'longer', 'than', 'necessary', 'aside', 'from', 'the', 'terrific', 'sea', 'rescue', 'sequences', 'of', 'which', 'there', 'are', 'very', 'few', 'i', 'just', 'did', 'not', 'care', 'about', 'any', 'of', 'the', 'characters', 'most', 'of', 'us', 'have', 'ghosts', 'in', 'the', 'closet', 'and', 'costner', 'character', 'are', 'realized', 'early', 'on', 'and', 'then', 'forgotten', 'until', 'much', 'later', 'by', 'which', 'time', 'i', 'did', 'not', 'care', 'the', 'character', 'we', 'should', 'really', 'care', 'about', 'is', 'a', 'very', 'cocky', 'overconfident', 'ashton', 'kutcher', 'the', 'problem', 'is', 'he', 'comes', 'off', 'as', 'kid', 'who', 'thinks', 'he', 'better', 'than', 'anyone', 'else', 'around', 'him', 'no', 'bed', 'and', 'shows', 'no', 'signs', 'of', 'a', 'cluttered', 'closet', 'his', 'only', 'obstacle', 'appears', 'to', 'be', 'winning', 'over', 'costner', 'lack', 'of', 'motiivation', 'finally', 'when

[['no', 'bed'],
 ['no', 'signs'],
 ['lack', 'of'],
 ['no', 'magic'],
 ['no', 'problem'],
 ['no', 'locks'],
 ['no', 'idea'],
 ['no', 'catalyst'],
 ['no', 'wonder'],
 ['no', 'man'],
 ['no', 'greta'],
 ['no', 'place'],
 ['no', 'suspense'],
 ['no', 'horror'],
 ['no', 'sense'],
 ['lack', 'of'],
 ['no', 'doubt'],
 ['no', 'reason'],
 ['no', 'intensity'],
 ['no', 'one'],
 ['no', 'brainer'],
 ['no', 'idea'],
 ['no', 'money'],
 ['no', 'chemistry'],
 ['no', 'one'],
 ['no', 'wonder'],
 ['no', 'actor'],
 ['no', 'matter'],
 ['no', 'one'],
 ['no', 'limit'],
 ['lack', 'of'],
 ['no', 'less'],
 ['no', 'horror'],
 ['lack', 'of'],
 ['no', 'sense'],
 ['no', 'idea'],
 ['no', 'one'],
 ['no', 'arc'],
 ['lack', 'of'],
 ['no', 'suspense'],
 ['no', 'enlightening'],
 ['no', 'shortage'],
 ['no', 'idea'],
 ['no', 'way'],
 ['no', 'purpose'],
 ['no', 'one'],
 ['lack', 'of'],
 ['no', 'questions'],
 ['lack', 'of'],
 ['no', 'resolution'],
 ['no', 'humor'],
 ['no', 'chemistry'],
 ['no', 'sizzle'],
 ['no', 'danger'],
 ['n

In [19]:
positive = []
for filename in os.listdir('pos_test'):
    pos = os.path.join('pos_test', filename)
    if os.path.isfile(pos) and filename.endswith('.txt'):
        with open(pos, 'r') as filepos:
            for inf in filepos:
                for slovo in nltk.word_tokenize(inf.lower()):
                    if slovo.isalpha():
                        finword = morph.parse(slovo)[0]
                        positive.append(finword[0])
print(positive)
chunker_pos(positive) #список из шаблона 3

['i', 'went', 'and', 'saw', 'this', 'movie', 'last', 'night', 'after', 'being', 'coaxed', 'to', 'by', 'a', 'few', 'friends', 'of', 'mine', 'i', 'admit', 'that', 'i', 'was', 'reluctant', 'to', 'see', 'it', 'because', 'from', 'what', 'i', 'knew', 'of', 'ashton', 'kutcher', 'he', 'was', 'only', 'able', 'to', 'do', 'comedy', 'i', 'was', 'wrong', 'kutcher', 'played', 'the', 'character', 'of', 'jake', 'fischer', 'very', 'well', 'and', 'kevin', 'costner', 'played', 'ben', 'randall', 'with', 'such', 'professionalism', 'the', 'sign', 'of', 'a', 'good', 'movie', 'is', 'that', 'it', 'can', 'toy', 'with', 'our', 'emotions', 'this', 'one', 'did', 'exactly', 'that', 'the', 'entire', 'theater', 'which', 'was', 'sold', 'out', 'was', 'overcome', 'by', 'laughter', 'during', 'the', 'first', 'half', 'of', 'the', 'movie', 'and', 'were', 'moved', 'to', 'tears', 'during', 'the', 'second', 'half', 'while', 'exiting', 'the', 'theater', 'i', 'not', 'only', 'saw', 'many', 'women', 'in', 'tears', 'but', 'many', '

[['well', 'trying'],
 ['well', 'developed'],
 ['well', 'developed'],
 ['well', 'fleshed'],
 ['well', 'done'],
 ['well', 'done'],
 ['well', 'acted'],
 ['well', 'made'],
 ['well', 'captured'],
 ['well', 'chosen'],
 ['well', 'done'],
 ['well', 'executed'],
 ['well', 'researched'],
 ['well', 'crafted'],
 ['well', 'done'],
 ['well', 'filmed']]

In [20]:
freqlist = Counter()
for word in positive:
    freqlist[word] += 1
freqlist_2 = Counter()
for word in negative:
    freqlist_2[word] += 1

In [22]:
freq_pos = {}
freq_neg = {}
for key, value in freqlist.items():
    if key not in freqlist_2.keys() and value != 1 and key != 'veronika' and key != 'veronica' and key != 'priest' and key!= 'nanny': #убрала те, которые встретились по одному разу (также Veronika, Veronica, priest, nanny), в прошлой домашке они убрались как имена собственные, либо первые слова предложений, потому что я делала по-другому
        freq_pos[key] = value
freq_pos = sorted(freq_pos.items(), key=lambda item: item[1], reverse=True)
print(dict(freq_pos))
for key, value in freqlist_2.items():
    if key not in freqlist.keys() and value != 1:
        freq_neg.update({key: value})
freq_neg = sorted(freq_neg.items(), key=lambda item: item[1], reverse=True)
print(dict(freq_neg))

{'wonderful': 33, 'thirst': 33, 'mehta': 28, 'merkerson': 26, 'blues': 23, 'lackawanna': 21, 'cranes': 18, 'soviet': 16, 'epatha': 16, 'ruben': 15, 'superb': 14, 'hyeon': 14, 'skeleton': 14, 'ms': 13, 'disney': 13, 'iwerks': 13, 'deepa': 12, 'strength': 12, 'masterpiece': 12, 'kim': 12, 'tatiana': 12, 'singing': 12, 'subtle': 11, 'bakjwi': 11, 'lesbian': 10, 'canadian': 10, 'desires': 10, 'outstanding': 10, 'twilight': 10, 'vampirism': 10, 'samojlova': 10, 'smits': 10, 'erotic': 9, 'rachel': 9, 'crosby': 9, 'macy': 9, 'hockey': 9, 'details': 8, 'freedom': 8, 'portrayed': 8, 'trilogy': 8, 'damned': 8, 'guardian': 8, 'training': 8, 'families': 8, 'magnificent': 8, 'lust': 8, 'tragedy': 8, 'mos': 8, 'def': 8, 'hbo': 8, 'tatyana': 8, 'era': 8, 'community': 8, 'jimmy': 8, 'boarding': 8, 'nero': 8, 'canadians': 8, 'marriages': 7, 'reminds': 7, 'wonderfully': 7, 'korean': 7, 'kalatozov': 7, 'russian': 7, 'law': 7, 'terrence': 7, 'circus': 7, 'alcohol': 7, 'maurice': 7, 'moved': 6, 'finest': 6

In [23]:
freq_lists = {}
freq_lists['pos'] = dict(freq_pos)
freq_lists['neg'] = dict(freq_neg)
print(freq_lists)

{'pos': {'wonderful': 33, 'thirst': 33, 'mehta': 28, 'merkerson': 26, 'blues': 23, 'lackawanna': 21, 'cranes': 18, 'soviet': 16, 'epatha': 16, 'ruben': 15, 'superb': 14, 'hyeon': 14, 'skeleton': 14, 'ms': 13, 'disney': 13, 'iwerks': 13, 'deepa': 12, 'strength': 12, 'masterpiece': 12, 'kim': 12, 'tatiana': 12, 'singing': 12, 'subtle': 11, 'bakjwi': 11, 'lesbian': 10, 'canadian': 10, 'desires': 10, 'outstanding': 10, 'twilight': 10, 'vampirism': 10, 'samojlova': 10, 'smits': 10, 'erotic': 9, 'rachel': 9, 'crosby': 9, 'macy': 9, 'hockey': 9, 'details': 8, 'freedom': 8, 'portrayed': 8, 'trilogy': 8, 'damned': 8, 'guardian': 8, 'training': 8, 'families': 8, 'magnificent': 8, 'lust': 8, 'tragedy': 8, 'mos': 8, 'def': 8, 'hbo': 8, 'tatyana': 8, 'era': 8, 'community': 8, 'jimmy': 8, 'boarding': 8, 'nero': 8, 'canadians': 8, 'marriages': 7, 'reminds': 7, 'wonderfully': 7, 'korean': 7, 'kalatozov': 7, 'russian': 7, 'law': 7, 'terrence': 7, 'circus': 7, 'alcohol': 7, 'maurice': 7, 'moved': 6, 'fi

#### Функция просто для деления по биграммам

In [24]:
def generate_ngrams(text, n=2):
    slova = text.split()
    output = []  
    for i in range(len(slova)- n+1):
        output.append(slova[i:i+n])
    return output

#### Функция для определения тональности отзыва + моя функция chunker

In [25]:
def tone_detect(freq_lists, text):
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[tone] += int(freq_list[word] > 0)
        for bigram in chunker_neg(negative):
            if bigram in generate_ngrams(text):
                counts['neg'] += 2              # я решила добавлять по два, так как как будто наличие биграмм в моих список - это важно
        for bigram in chunker_pos(positive):
            if bigram in generate_ngrams(text):
                counts['pos'] += 2
    return counts.most_common()

#### Проверка точности нового алгоритма

In [26]:
results = []
check = []
for filename in os.listdir('check'):
    f = os.path.join('check', filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f, 'r') as file:
            for text in file:
                predicted_tone = tone_detect(freq_lists, text)[0][0]
                results.append(predicted_tone)
                check.append(filename[:3])
print(results)
print(check)
print("Accuracy: %.4f" % accuracy_score(results, check))

['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'neg', 'pos', 'neg', 'pos', 'pos', 'neg']
['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
Accuracy: 0.7826


#### Функция определения тональности из дз 1 без биграмм

In [27]:
def old_tone_detect(freq_lists, text):
    counts = Counter()
    for tone, freq_list in freq_lists.items():
        freq_list = Counter(freq_list)
        for word in nltk.word_tokenize(text):
            counts[tone] += int(freq_list[word] > 0)
    return counts.most_common()

#### Старая проверка точности из дз1

In [29]:
old_results = []
check = []
for filename in os.listdir('check'):
    f = os.path.join('check', filename)
    if os.path.isfile(f) and filename.endswith('.txt'):
        with open(f, 'r') as file:
            for text in file:
                predicted_tone = old_tone_detect(freq_lists, text)[0][0]
                old_results.append(predicted_tone)
                check.append(filename[:3])
print(old_results)
print(check)
print("Accuracy: %.4f" % accuracy_score(old_results, check))

['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'neg']
['neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'neg', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos', 'pos']
Accuracy: 0.8696


## вывод: хуже работает функция c биграммами...